In [ ]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

import symforce.symbolic as sf
from symforce.ops import LieGroupOps
from symforce.ops import StorageOps

epsilon = 1e-9

In [ ]:
def storage_D_tangent(a):
    """
    Computes the jacobian of the storage space of an element with respect to the tangent space around
    that element.

    This is exact, but can be slow because we take a symbolic limit.
    """
    # Perturb a in the tangent space
    tangent_dim = LieGroupOps.tangent_dim(a)
    xi = sf.Matrix(tangent_dim, 1).symbolic("xi")
    a_perturbed = LieGroupOps.retract(a, xi)
    a_perturbed_storage = sf.Matrix(StorageOps.to_storage(a_perturbed))

    # Compute jacobian of storage wrt perturbation
    storage_D_tangent = a_perturbed_storage.jacobian(xi)

    # Take the limit as perturbation goes to zero
    # NOTE: This can be very slow
    for x in xi:
        storage_D_tangent = storage_D_tangent.limit(x, 0)

    return storage_D_tangent

In [ ]:
def storage_D_tangent_approx(a, epsilon):
    """
    Computes the jacobian of the storage space of an element with respect to the tangent space around
    that element.

    This is an approximation, but is much faster than storage_D_tangent. Note that the exact jacobian
    can often be recovered with a call to nsimplify with the appropriate tolerance (though this requires
    the use of sympy rather than symengine)
    """
    # Perturb a in the tangent space
    tangent_dim = LieGroupOps.tangent_dim(a)
    xi = sf.Matrix(tangent_dim, 1).symbolic("xi")
    a_perturbed = LieGroupOps.retract(a, xi)
    a_perturbed_storage = sf.Matrix(StorageOps.to_storage(a_perturbed))

    # Compute jacobian of storage wrt perturbation
    storage_D_tangent = a_perturbed_storage.jacobian(xi)

    # Rather than computing the limit, we substitude a small value for xi to approximate the limit
    # NOTE: This is much faster than taking the limit in sympy, but returns an approximation of the true
    # jacobian.
    assert epsilon != 0
    storage_D_tangent = storage_D_tangent.subs(xi, epsilon * xi.one())

    return storage_D_tangent

In [ ]:
matrix = sf.Matrix23.symbolic("self")
display(storage_D_tangent_approx(matrix, epsilon))
display(storage_D_tangent(matrix))

In [ ]:
rot2 = sf.Rot2.symbolic("self")
display(storage_D_tangent_approx(rot2, epsilon))
display(storage_D_tangent(rot2))

In [ ]:
rot3 = sf.Rot3.symbolic("self")
display(storage_D_tangent_approx(rot3, epsilon))
display(storage_D_tangent(rot3))  # This will take a while

In [ ]:
pose2 = sf.Pose2.symbolic("self")
display(storage_D_tangent_approx(pose2, epsilon))
display(storage_D_tangent(pose2))

In [ ]:
pose3 = sf.Pose3.symbolic("self")
display(storage_D_tangent_approx(pose3, epsilon))
display(storage_D_tangent(pose3))  # This will take a while

In [ ]:
unit3 = sf.Unit3.symbolic("self")
display(storage_D_tangent_approx(unit3, epsilon))
display(storage_D_tangent(unit3))